# Pairings

In [1]:
# Loading the Pairing from external file 
load("bls-pairing.sage")
# Hashing algo
from hashlib import sha256

## Playing with pairings

In [2]:
# 

# We have Curve and a twisted version of it
# E1 is the original curve over F1
# E12 is the original curve over F12 - 12th extension of the original F1
# E2 is a twisted E12, where we reduced the original field to reduce computation complexity

# G1 and G2 are generators of the E1 and E2
# To construct a pairing we need to lift both points to the E12 and do it there
# Lifting is same as making sure the points are in the right group 

assert BLSpairing.pair(G1 * 3, G2 * 2) == BLSpairing.pair(G1, G2)^6

## BLS and EcDSA Signatures
### Setup

In [3]:
G1F = GF(G1.order()) # element of Cyclic group of order eqaul to order of G1 in E1

In [4]:
def elev_hash_to_point(hashI, E):
    
    hashF = F2(hashI)
            
    while True:
        
        try:
            hashP = E.x_lift(hashF) # point on the E2
        except:
            hashF += 1
    
    hashG2P = h2 * hashP # point on the E2, beloning to the G2 group (multiply by cofactor to get this)
    
    return hashG2P


def hash_to_point(hashI, E):
    
    hashF = F2(hashI)
    
    while True:
        
        hashG2P = G2 * hashF # This is an alternitive way by mutliplying G2 by the hashF - point in G2 on E2
        
        if hashG2P != 0:
            break
            
        hashF += 1
        
    return hashG2P 
    

# Hash the message with SHA256, return an integer 
def hash256(m):
    
    hex_hash = sha256(str(m).encode("utf-8")).hexdigest()
    
    return int(hex_hash, 16)

In [45]:
class Signer:
    def __init__(self):
        self.prk = G1F.random_element() # element of G1F
        self.pbk = self.prk * G1 # Point of E1 generated by G1 
    
    def get_pbk(self):
        return self.pbk
    
    def naive_sign(self, m):
        
        m_hash = G1F(hash256(m)) # element of G1F
        
        
        return m_hash * self.prk # element of G1F
        
        
    def ecdsa_sgn(self, m):
        
        m_hash = G1F(hash256(m))
        
        while True:
            k = G1F.random_element() # element of G1F
            
            x1 = (k * G1).xy()[0] # X coordinate of E1 Point, F1
            
            r = G1F(x1) # F1 element, then converted to G1F
            
            if r == 0: # If r = 0, we do not have a prk info in the signature -> try another k
                continue
                
            sgn = (~k) * (m_hash + r * self.prk)  # element of G1F
            
            if sgn != 0: # If sgn = 0, we do not have any info in the signature -> try another k
                break
                
        return (r, sgn)           
    
        
    
    def bls_sgn(self, m):
        
        m_point = hash_to_point(hash256(m), E2) # point on the E2 in the G2
        
        return self.prk * m_point # point on the E2 in the G2 
    


In [56]:
class Verifier:
    
    @staticmethod
    def verify_signature(sgn, r):

        assert(r > 0 and r < G1.order(), "r must be bounded in [1 to n - 1]")
        assert(sgn > 0 and sgn < G1.order(), "r must be bounded in [1 to n - 1]")
        
        
    @staticmethod
    def verify_public_key(pbk):
        
        assert(pbk != 0, "Public key can not be null")
        assert(pbk * G1.order() == 0, "Public key is incorrect, not generated from G1")

        
    
    @staticmethod
    def naive_sgn_check(m, pbk, sgn):
        
        m_hash = G1F(hash256(m)) # element of G1F
        
        assert(m_hash * pbk == sgn * G1)
        print("Naive verification passed for pbk {}, Message `{}`".format(pbk.xy()[0], m))
        
        
    @classmethod
    def ecdsa_sgn_check(cls, m, pbk, r_sgn):
        
        r, sgn = r_sgn
           
        cls.verify_public_key(pbk)
        cls.verify_signature(sgn, r)
        
        
        m_hash = G1F(hash256(m)) # element of G1F
        
        u1 = (~sgn) * m_hash # element of G1F
        u2 = (~sgn) * r  # element of G1F
        
        res = (u1 * G1 + u2 * pbk).xy()[0] # X coordinate of E1 Point, F1
        
        assert(G1F(res) == r) # Convert F1 to G1F
        print("ECDSA verification passed.")
        
        
        
        
    @classmethod
    def ecdsa_pbk_extract(cls, m, r_sgn):
        
        # We assume that the signature is correct, as otherwise we would recover an incorrect key 
        # or fail to recover the key all together
        
        r, sgn = r_sgn
        
        cls.verify_signature(sgn, r) 
        
        m_hash = G1F(hash256(m)) # element of G1F
        
        valid_points = []
        r_x = F1(r) # x coordinate of R on the E1, element of F1
        for i in range(0, p):
            
            valid_points = E1.lift_x(r_x, all=True)

            if len(valid_points) == 0:
                continue

    
        assert(len(valid_points) != 0, "No valid public key has been found for the signature!")
        
        
        u1 = (-1 * m_hash) * (~r) # element of G1F
        u2 = sgn * (~r) # element of G1F
        
        return [(u1 * G1 + u2 * point) for point in valid_points]   
    
    
    @classmethod
    def bls_sgn_check(cls, m, pbk, sgn):
        
        assert (sgn != 0, "Signature must not be Null.")
        
        cls.verify_public_key(pbk)
        
        m_point = hash_to_point(hash256(m), E2) # point on the E2 in the G2
                
        assert(BLSpairing.pair(G1, sgn) == BLSpairing.pair(pbk, m_point))
        print("BLS verification passed.")
        
        
    @classmethod
    def bls_sgn_agr_check(cls, m, pbk_arr, sgn_arr):
        
        # We assume that we have already checked that the public keys have a corresponding private key
        # Otherwise we open up an attack vector where an attacker can make it look like we have signed by providing a rouge signature
        
        assert(len(pbk_arr) != 0, "Not an empty list of public keys must be provided.")
        assert(len(sgn_arr) != 0, "Not an empty list of signatures must be provided.")
        
        # Check not used as we might be getting a aggregated signatures / public keys already
        # assert(len(pbk_arr) == len(sgn_arr), "Amount of signatures must be equal to amount of public keys.")
        
        for sgn in sgn_arr:
            assert (sgn != 0, "Signature must not be Null.")
            
        for pbk in pbk_arr:
            cls.verify_public_key(pbk)
            
            
        m_point = hash_to_point(hash256(m), E2) # point on the E2 in the G2
            
            
        pbk_agr = reduce((lambda pbk_pagr, pbk: pbk_pagr + pbk), pbk_arr)
        sgn_agr = reduce((lambda sgn_pagr, sgn: sgn_pagr + sgn), sgn_arr)
        
        
        assert(BLSpairing.pair(G1, sgn_agr) == BLSpairing.pair(pbk_agr, m_point))
        print("BLS Agr verification passed.")
        
      
    # Function to check that m messages have been signed correctly by n parties
    # Requires m + 1 pairing to verify
    @classmethod
    def bls_mm_sgn_agr_check(cls, m_arr, pbk_arr, sgn_arr):
        
        # We assume that we have already checked that the public keys have a corresponding private key
        # Otherwise we open up an attack vector where an attacker can make it look like we have signed by providing a rouge signature
        
        assert(len(pbk_arr) != 0, "Not an empty list of public keys must be provided.")
        assert(len(sgn_arr) != 0, "Not an empty list of signatures must be provided.")
        
        
        for sgn in sgn_arr:
            assert (sgn != 0, "Signature must not be Null.")
            
        for pbk in pbk_arr:
            cls.verify_public_key(pbk)
            
            
        m_point_arr = [hash_to_point(hash256(m), E2) for m in m_arr] # Convert all messages to G2 EC points 
        
        pbk_agr = reduce((lambda pbk_pagr, pbk: pbk_pagr + pbk), pbk_arr)
        sgn_agr = reduce((lambda sgn_pagr, sgn: sgn_pagr + sgn), sgn_arr)


        sgn_agr_pairing = BLSpairing.pair(G1, sgn_agr)
        
        
        m_pairing_arr = [BLSpairing.pair(pbk_agr, m_point) for m_point in m_point_arr]
        
        
        assert(sgn_agr_pairing == reduce((lambda pairing_agr, pairing: pairing_agr * pairing), m_pairing_arr))
        print("BLS MM Agr verification passed.")

### EcDSA and BLS Signature Example

In [57]:
signer = Signer()

m = "hey!"

In [58]:
# Sign the message in two ways
ecdsa_sgn = signer.ecdsa_sgn(m)
bls_sgn = signer.bls_sgn(m)
pbk = signer.get_pbk()

In [59]:
# Verify the two signatures
Verifier.ecdsa_sgn_check(m, pbk, ecdsa_sgn)
Verifier.bls_sgn_check(m, pbk, bls_sgn)

ECDSA verification passed.
BLS verification passed.


In [61]:
# Attempt to extract the EcDSA signature
# *Verifier.ecdsa_pbk_extract(m,  ecdsa_sgn)

###  Message & Signature Aggregation Example

In [62]:
signer_n = 5
message_n = 3

m = "Everyone must sign!"
m_arr = ["Message " + str(i) for i in range(1, message_n)]

signers = [Signer() for i in range(1, signer_n)]

In [63]:
# Create a list of signatures over the m and m_arr
sgn_arr = [signer.bls_sgn(m) for signer in signers]
arr_sgn_arr = [signer.bls_sgn(m_i) for signer in signers for m_i in m_arr]
pbk_arr = [signer.get_pbk() for signer in signers]

In [64]:
# Check the aggregate signatures
Verifier.bls_sgn_agr_check(m, pbk_arr, sgn_arr)
Verifier.bls_mm_sgn_agr_check(m_arr, pbk_arr, arr_sgn_arr)

BLS Agr verification passed.
BLS MM Agr verification passed.
